In [1]:
%cd  /home/wanddy/ChatGLM-Tuning/
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:32"
os.environ["CUDA_VISIBLE_DEVICES"] = "6"

/home/wanddy/ChatGLM-Tuning


In [2]:

from modeling_chatglm import ChatGLMForConditionalGeneration
import torch


torch.set_default_tensor_type(torch.cuda.HalfTensor)
model = ChatGLMForConditionalGeneration.from_pretrained("chatglm-6b", trust_remote_code=True, device_map='auto')

/home/wanddy/ChatGLM-6B/.conda/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
Loading checkpoint shards: 100%|██████████| 8/8 [00:12<00:00,  1.54s/it]


In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("chatglm-6b", trust_remote_code=True)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


In [4]:
from peft import get_peft_model, LoraConfig, TaskType

peft_path = "output/adapter_model.bin"

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, inference_mode=True,
    r=8,
    lora_alpha=32, lora_dropout=0.1
)

model = get_peft_model(model, peft_config)
model.load_state_dict(torch.load(peft_path), strict=False)
torch.set_default_tensor_type(torch.cuda.FloatTensor)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /home/wanddy/ChatGLM-6B/.conda/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/wanddy/ChatGLM-6B/.conda/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/home/wanddy/ChatGLM-6B/.conda/lib/python3.9/site-packages/peft/tuners/lora.py:175: UserWarning: fan_in_fan_out is set to True but the target module is not a Conv1D. Setting fan_in_fan_out to False.
  warnings.warn(


In [5]:
import json

instructions = json.load(open("data/alpaca_data.json",encoding="utf-8"))

In [6]:
answers = []
from cover_alpaca2jsonl import format_example


with torch.no_grad():
    for idx, item in enumerate(instructions[:5]):
        feature = format_example(item)
        input_text = feature['context']
        ids = tokenizer.encode(input_text)
        input_ids = torch.LongTensor([ids])
        out = model.generate(
            input_ids=input_ids,
            max_length=150,
            do_sample=False,
            temperature=0
        )
        out_text = tokenizer.decode(out[0])
        answer = out_text.replace(input_text, "").replace("\nEND", "").strip()
        item['infer_answer'] = answer
        print(out_text)
        print(f"### {idx+1}.Answer:\n", item.get('output'), '\n\n')
        answers.append({'index': idx, **item})

/home/wanddy/ChatGLM-6B/.conda/lib/python3.9/site-packages/transformers/generation/utils.py:1374: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Instruction: 2023年达州市市委书记是谁?
Answer: 2023年达州市市委书记是邵革军。
### 1.Answer:
 2023年达州市市委书记是邵革军。 


Instruction: 22023年达州市市长是谁?
Answer: 22023年达州市市长是严卫东。
### 2.Answer:
 2023年达州市市长是严卫东。 


Instruction: 什么是三原色?
Answer: 三原色是红、蓝、黄。
### 3.Answer:
 三原色是红、蓝、黄。 


Instruction: 描述原子的结构。
Answer: 原子由带正电荷的质子和带负电荷的电子组成。质子位于原子核中心,而电子围绕原子核运动。原子核由带正电荷的质子和带负电荷的中子组成。
### 4.Answer:
 原子由原子核组成，原子核包含质子和中子，周围环绕着围绕原子核运行的电子。质子和中子带正电荷，而电子带负电荷，从而形成整体中性原子。每个粒子的数量决定了原子序数和原子类型。 


Instruction: 我们怎样才能减少空气污染?
Answer: 减少空气污染的方法包括使用清洁能源、限制汽车使用、鼓励公共交通、鼓励骑自行车、鼓励步行和限制工业和运输活动。此外,政府可以通过实施法规和限制来减少空气污染。
### 5.Answer:
 减少空气污染的方法有很多，例如转向可再生能源、鼓励使用公共交通工具、禁止燃烧化石燃料、实施工业源减排政策以及实施机动车排放标准。此外，个人可以通过减少汽车使用、避免燃烧木材等材料以及改用节能电器来为减少空气污染做出自己的贡献。 




In [7]:
history = []
print("欢迎使用 ChatGLM-6B 模型，输入内容即可进行对话，clear 清空对话历史，stop 终止程序")
while True:
    query = input("\n用户：")
    if query == "stop":
        break
    if query == "clear":
        history = []        
        os.system(query)
        print("欢迎使用 ChatGLM-6B 模型，输入内容即可进行对话，clear 清空对话历史，stop 终止程序")
        continue
    response, history = model.chat(tokenizer, query, history=history)
    print(f"ChatGLM-6B：{response}")

欢迎使用 ChatGLM-6B 模型，输入内容即可进行对话，clear 清空对话历史，stop 终止程序
